In [1]:
!pip install -U sentence-transformers matplotlib seaborn



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Found credentials from IAM Role: cld_d81e97c8fg46h1crfqpx4w37ei-cluster-node-role


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch

%matplotlib inline

In [9]:
path1='/home/ray/default/synthetic_sentence_med_nodp_temp1.2_topk50.csv'
#path2="/home/ray/default/synthetic_data_sst2_eps3.csv"
df = pd.read_csv(path1)
print(df.shape)
df.head()

(2014, 19)


,transcription,medical_specialty,age_related_sentence,extracted_text,med_masked_transcriptions,pii_masked_transcriptions,synthetic_sentence_nodp,synthetic_sentence2,synthetic_sentence_dp_nobossep,synthetic_sentence_dp_eps16,synthetic_sentence_dp_eps8_v2,synthetic_sentence_dp_eps3,synthetic_sentence_nodp_temp1,synthetic_sentence_med_nodp_temp0.6_topk100,synthetic_sentence_med_nodp_temp0.8_topk100,synthetic_sentence_med_nodp_temp1.2_topk100,synthetic_sentence_med_nodp_temp0.6_topk50,synthetic_sentence_med_nodp_temp0.8_topk50,synthetic_sentence_med_nodp_temp1.2_topk50
0,"CC:, Confusion and slurred speech.,HX , (prima...",Neurology,"HX , (primarily obtained from boyfriend): This...","HX , (primarily obtained from boyfriend): This...","HX , (primarily obtained from boyfriend): This...","HX , (primarily obtained from boyfriend): This...",[BOS]Neurology[SEP]The patient is an approxima...,"[BOS]Neurology[SEP] Ne ne]D D (N,D)Ne (D,N)D (...","Specialty: Neurology presentation, she was fo...",[BOS]Neurology[SEP]This patient has uctal dysf...,[BOS]Neurology[SEP]This patient has epidural ...,"[BOS]Neurology[SEP]This patient has SEPRA, a p...",[BOS]Neurology[SEP]This patient has This is a ...,[BOS]Neurology[SEP]This patient has The patien...,[BOS]Neurology[SEP]This patient has 53-year-o...,[BOS]Neurology[SEP]This patient has The risks ...,"[BOS]Neurology[SEP]This patient has ,The patie...",[BOS]Neurology[SEP]This patient has He had a ...,[BOS]Neurology[SEP]This patient has The patie...
1,"PREOPERATIVE DIAGNOSES,Airway obstruction seco...",Cardiovascular / Pulmonary,"INDICATIONS FOR SURGERY,The patient is a 50-ye...","INDICATIONS FOR SURGERY,The patient is a 50-ye...","INDICATIONS FOR SURGERY,The patient is a[AGE] ...","INDICATIONS FOR SURGERY,The patient is a[AGE] ...",[BOS]Cardiovascular / Pulmonary[SEP]The patien...,[BOS]Cardiovascular / Pulmonary[SEP] heart car...,Specialty: Cardiovascular / Pulmonary ressive ...,[BOS]Cardiovascular / Pulmonary[SEP]This patie...,[BOS]Cardiovascular / Pulmonary[SEP]This patie...,[BOS]Cardiovascular / Pulmonary[SEP]This patie...,[BOS]Cardiovascular / Pulmonary[SEP]This patie...,[BOS]Cardiovascular / Pulmonary[SEP]This patie...,[BOS]Cardiovascular / Pulmonary[SEP]This patie...,[BOS]Cardiovascular / Pulmonary[SEP]This patie...,[BOS]Cardiovascular / Pulmonary[SEP]This patie...,[BOS]Cardiovascular / Pulmonary[SEP]This patie...,[BOS]Cardiovascular / Pulmonary[SEP]This patie...
2,"PROCEDURE: , Elective male sterilization via b...",Urology,"INDICATIONS: ,This 34-year-old gentleman has ...",This 34-year-old gentleman has come to the off...,This[AGE][SEX] has come to the office requesti...,This[AGE][SEX] has come to the office requesti...,"[BOS]Urology[SEP]This is an 18-year-old male, ...",[BOS]Urology[SEP]Puurology [SEuep]QiQiiQivQeiq...,Specialty: Urology the need for procedure wit...,[BOS]Urology[SEP]This patient has opedic compl...,[BOS]Urology[SEP]This patient has peritoneopil...,[BOS]Urology[SEP]This patient has ergosteretri...,[BOS]Urology[SEP]This patient has The patient...,[BOS]Urology[SEP]This patient has The patient...,[BOS]Urology[SEP]This patient has The patient...,[BOS]Urology[SEP]This patient has The procedu...,[BOS]Urology[SEP]This patient has a history o...,[BOS]Urology[SEP]This patient has Patient is ...,[BOS]Urology[SEP]This patient has the patient...
3,"DESCRIPTION:, The patient was placed in the s...",Urology,"DESCRIPTION:, The patient was placed in the s...",The patient was placed in the supine position ...,The patient was placed in the supine position ...,The patient was placed in the supine position ...,"[BOS]Urology[SEP]After the procedure was over,...",[BOS]Urology[SEP]MM)\nN\n\nM\nS\nN,Specialty: Urology ft vas was grasped in betwe...,[BOS]Urology[SEP]This patient is iphgym who ha...,[BOS]Urology[SEP]This patient is ialectonic pr...,[BOS]Urology[SEP]This patient is SEP. The pati...,"[BOS]Urology[SEP]This patient is ,Patient pres...",[BOS]Urology[SEP]This patient is On this pati...,[BOS]Ur

## Clean up Seperators from Synthetic Data Generation

In [10]:
import pandas as pd
import re

def remove_before_sep(df, column_name):
    """
    Removes everything before '[SEP]' in the specified column of the DataFrame.

    Parameters:
    - df: DataFrame containing the data.
    - column_name: The name of the column to process.

    Returns:
    - df: DataFrame with the processed column.
    """
    # Define the regular expression pattern to match everything before '[SEP]'
    pattern = re.compile(r'.*?\[SEP\]')

    # Apply the regular expression to the specified column
    df[column_name] = df[column_name].apply(lambda x: re.sub(pattern, '', x))

    return df

## SST 2 Cleaning 

In [7]:
# Remove everything before '[SEP]' in the 'synthetic_sentence2' column
df = remove_before_sep(df, 'synthetic_sentence_nodp')
df = remove_before_sep(df, 'synthetic_sentence_dp_eps16')
df = remove_before_sep(df, 'synthetic_sentence_dp_eps8')
df = remove_before_sep(df, 'synthetic_sentence_dp_eps3')
df = remove_before_sep(df, 'synthetic_sentence_nodp_temp0.6_topk100')
df = remove_before_sep(df, 'synthetic_sentence_nodp_temp1.2_topk100')
df = remove_before_sep(df, 'synthetic_sentence_nodp_temp0.8_topk50')

## Medical Data Cleaning

In [11]:
df.columns

Index(['transcription', 'medical_specialty', 'age_related_sentence',
       'extracted_text', 'med_masked_transcriptions',
       'pii_masked_transcriptions', 'synthetic_sentence_nodp',
       'synthetic_sentence2', 'synthetic_sentence_dp_nobossep',
       'synthetic_sentence_dp_eps16', 'synthetic_sentence_dp_eps8_v2',
       'synthetic_sentence_dp_eps3', 'synthetic_sentence_nodp_temp1',
       'synthetic_sentence_med_nodp_temp0.6_topk100',
       'synthetic_sentence_med_nodp_temp0.8_topk100',
       'synthetic_sentence_med_nodp_temp1.2_topk100',
       'synthetic_sentence_med_nodp_temp0.6_topk50',
       'synthetic_sentence_med_nodp_temp0.8_topk50',
       'synthetic_sentence_med_nodp_temp1.2_topk50'],
      dtype='object')

In [12]:
# Remove everything before '[SEP]' in the specified columns
df = remove_before_sep(df, 'synthetic_sentence_nodp')
df = remove_before_sep(df, 'synthetic_sentence_dp_eps16')
df = remove_before_sep(df, 'synthetic_sentence_dp_eps8_v2')
df = remove_before_sep(df, 'synthetic_sentence_dp_eps3')
df = remove_before_sep(df, 'synthetic_sentence_nodp_temp1')
df = remove_before_sep(df, 'synthetic_sentence_med_nodp_temp0.6_topk100')
df = remove_before_sep(df, 'synthetic_sentence_med_nodp_temp0.8_topk100')
df = remove_before_sep(df, 'synthetic_sentence_med_nodp_temp1.2_topk100')
df = remove_before_sep(df, 'synthetic_sentence_med_nodp_temp0.6_topk50')
df = remove_before_sep(df, 'synthetic_sentence_med_nodp_temp0.8_topk50')
df = remove_before_sep(df, 'synthetic_sentence_med_nodp_temp1.2_topk50')

## Calculate Fidelity based on Cosine Similarity

In [13]:
import pandas as pd
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

def calculate_fidelity(df, column_original, column_synthetic):
    # Load the SimCSE model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
    model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

    # Extract sentences from the DataFrame columns
    original_sentences = df[column_original].dropna().tolist()
    synthetic_sentences = df[column_synthetic].dropna().tolist()

    # Function to get embeddings for a list of sentences
    def get_embeddings(sentences):
        inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.pooler_output  # Use pooler output for SimCSE
        return embeddings

    # Compute embeddings for the original and synthetic sentences
    original_embeddings = get_embeddings(original_sentences)
    synthetic_embeddings = get_embeddings(synthetic_sentences)

    # Calculate pairwise cosine similarity
    similarity_scores = [
        1 - cosine(original_embeddings[i], synthetic_embeddings[i])
        for i in range(len(original_embeddings))
    ]

    # Example aggregation: Calculate the mean of the similarity scores
    mean_similarity = sum(similarity_scores) / len(similarity_scores)

    return mean_similarity


### SST 2 Fidelity

In [14]:
df.columns

Index(['transcription', 'medical_specialty', 'age_related_sentence',
       'extracted_text', 'med_masked_transcriptions',
       'pii_masked_transcriptions', 'synthetic_sentence_nodp',
       'synthetic_sentence2', 'synthetic_sentence_dp_nobossep',
       'synthetic_sentence_dp_eps16', 'synthetic_sentence_dp_eps8_v2',
       'synthetic_sentence_dp_eps3', 'synthetic_sentence_nodp_temp1',
       'synthetic_sentence_med_nodp_temp0.6_topk100',
       'synthetic_sentence_med_nodp_temp0.8_topk100',
       'synthetic_sentence_med_nodp_temp1.2_topk100',
       'synthetic_sentence_med_nodp_temp0.6_topk50',
       'synthetic_sentence_med_nodp_temp0.8_topk50',
       'synthetic_sentence_med_nodp_temp1.2_topk50'],
      dtype='object')

In [15]:
# Calculate fidelity for the different comparisons
fidelity_sentence_extracted = calculate_fidelity(df, 'sentence', 'sentence')
fidelity_sentence_synthetic_nodp = calculate_fidelity(df, 'sentence', 'synthetic_sentence_nodp')
fidelity_sentence_synthetic_dp_eps16 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_dp_eps16')
fidelity_sentence_synthetic_dp_eps8 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_dp_eps8')
fidelity_sentence_synthetic_dp_eps3 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_dp_eps3')
fidelity_sentence_synthetic_nodp_temp06_topk100 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_nodp_temp0.6_topk100')
fidelity_sentence_synthetic_nodp_temp12_topk100 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_nodp_temp1.2_topk100')
fidelity_sentence_synthetic_nodp_temp08_topk50 = calculate_fidelity(df, 'sentence', 'synthetic_sentence_nodp_temp0.8_topk50')

# Print the results
print(f"Fidelity (Sentence vs Extracted Text): {fidelity_sentence_extracted:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence No DP): {fidelity_sentence_synthetic_nodp:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence DP Epsilon 16): {fidelity_sentence_synthetic_dp_eps16:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence DP Epsilon 8): {fidelity_sentence_synthetic_dp_eps8:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence DP Epsilon 3): {fidelity_sentence_synthetic_dp_eps3:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence No DP Temp 0.6 Top K 100): {fidelity_sentence_synthetic_nodp_temp06_topk100:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence No DP Temp 1.2 Top K 100): {fidelity_sentence_synthetic_nodp_temp12_topk100:.3f}")
print(f"Fidelity (Sentence vs Synthetic Sentence No DP Temp 0.8 Top K 50): {fidelity_sentence_synthetic_nodp_temp08_topk50:.3f}")


KeyError: 'sentence'

## Medical Data Fidelity Calculation

In [ ]:
# Calculate fidelity for various comparisons
fidelity_extracted_extracted = calculate_fidelity(df, 'extracted_text', 'extracted_text')
fidelity_extracted_pii_masked = calculate_fidelity(df, 'extracted_text', 'pii_masked_transcriptions')
fidelity_extracted_synthetic = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_nodp')
fidelity_extracted_syntheticeps8 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_eps8_v2')
fidelity_extracted_syntheticeps16 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_eps16')
fidelity_extracted_synthetic_dp_nobossep = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_nobossep')

# Print the results
print(f"Fidelity (Extracted Text vs Extracted Text): {fidelity_extracted_extracted:.3f}")
print(f"Fidelity (Extracted Text vs PII Masked Transcriptions): {fidelity_extracted_pii_masked:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence No DP): {fidelity_extracted_synthetic:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence DP eps8): {fidelity_extracted_syntheticeps8:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence DP eps16): {fidelity_extracted_syntheticeps16:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence DP No BOS SEP): {fidelity_extracted_synthetic_dp_nobossep:.3f}")

In [8]:
df.columns

Index(['transcription', 'medical_specialty', 'age_related_sentence',
       'extracted_text', 'med_masked_transcriptions',
       'pii_masked_transcriptions', 'synthetic_sentence_nodp',
       'synthetic_sentence2', 'synthetic_sentence_dp_nobossep',
       'synthetic_sentence_dp_eps16', 'synthetic_sentence_dp_eps8_v2',
       'synthetic_sentence_dp_eps3', 'synthetic_sentence_nodp_temp1',
       'synthetic_sentence_med_nodp_temp0.6_topk100',
       'synthetic_sentence_med_nodp_temp0.8_topk100',
       'synthetic_sentence_med_nodp_temp1.2_topk100'],
      dtype='object')

In [16]:
#Calculate fidelity for various comparisons
fidelity_extracted_extracted = calculate_fidelity(df, 'extracted_text', 'extracted_text')
fidelity_extracted_pii_masked = calculate_fidelity(df, 'extracted_text', 'pii_masked_transcriptions')
fidelity_extracted_synthetic_temp1 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_nodp_temp1')
fidelity_extracted_synthetic_temp0_6_topk100 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_med_nodp_temp0.6_topk100')
fidelity_extracted_synthetic_temp0_8_topk100 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_med_nodp_temp0.8_topk100')
fidelity_extracted_synthetic_temp1_2_topk100 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_med_nodp_temp1.2_topk100')
fidelity_extracted_synthetic_temp0_6_topk50 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_med_nodp_temp0.6_topk50')
fidelity_extracted_synthetic_temp0_8_topk50 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_med_nodp_temp0.8_topk50')
fidelity_extracted_synthetic_temp1_2_topk50 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_med_nodp_temp1.2_topk50')

#Print the results
print(f"Fidelity (Extracted Text vs Extracted Text): {fidelity_extracted_extracted:.3f}")
print(f"Fidelity (Extracted Text vs PII Masked Transcriptions): {fidelity_extracted_pii_masked:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence Temp 1 No DP): {fidelity_extracted_synthetic_temp1:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence Temp 0.6 TopK 100): {fidelity_extracted_synthetic_temp0_6_topk100:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence Temp 0.8 TopK 100): {fidelity_extracted_synthetic_temp0_8_topk100:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence Temp 1.2 TopK 100): {fidelity_extracted_synthetic_temp1_2_topk100:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence Temp 0.6 TopK 50): {fidelity_extracted_synthetic_temp0_6_topk50:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence Temp 0.8 TopK 50): {fidelity_extracted_synthetic_temp0_8_topk50:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence Temp 1.2 TopK 50): {fidelity_extracted_synthetic_temp1_2_topk50:.3f}")


Fidelity (Extracted Text vs Extracted Text): 1.000
Fidelity (Extracted Text vs PII Masked Transcriptions): 0.929
Fidelity (Extracted Text vs Synthetic Sentence Temp 1 No DP): 0.523
Fidelity (Extracted Text vs Synthetic Sentence Temp 0.6 TopK 100): 0.525
Fidelity (Extracted Text vs Synthetic Sentence Temp 0.8 TopK 100): 0.527
Fidelity (Extracted Text vs Synthetic Sentence Temp 1.2 TopK 100): 0.526
Fidelity (Extracted Text vs Synthetic Sentence Temp 0.6 TopK 50): 0.523
Fidelity (Extracted Text vs Synthetic Sentence Temp 0.8 TopK 50): 0.525
Fidelity (Extracted Text vs Synthetic Sentence Temp 1.2 TopK 50): 0.524


In [10]:
# Calculate fidelity for the three different comparisons
fidelity_extracted_extracted = calculate_fidelity(df, 'extracted_text', 'extracted_text')
fidelity_extracted_pii_masked = calculate_fidelity(df, 'extracted_text', 'pii_masked_transcriptions')
fidelity_extracted_synthetic = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_nodp')

# Print the results
print(f"Fidelity (Extracted Text vs Extracted Text): {fidelity_extracted_extracted:.3f}")
print(f"Fidelity (Extracted Text vs PII Masked Transcriptions): {fidelity_extracted_pii_masked:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence No DP): {fidelity_extracted_synthetic:.3f}")

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

/home/ray/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x78babcfff580>>
Traceback (most recent call last):
  File "/home/ray/anaconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyError: 'extracted_text'

In [12]:
df.columns

Index(['transcription', 'medical_specialty', 'age_related_sentence',
       'extracted_text', 'med_masked_transcriptions',
       'pii_masked_transcriptions', 'synthetic_sentence_nodp',
       'synthetic_sentence2', 'synthetic_sentence_dp_nobossep',
       'synthetic_sentence_dp_eps16', 'synthetic_sentence_dp_eps8_v2'],
      dtype='object')

In [11]:
fidelity_extracted_syntheticeps8 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_eps8_v2')
#fidelity_extracted_syntheticeps16 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_eps16')

print(f"Fidelity (Extracted Text vs Synthetic Sentence DP eps8): {fidelity_extracted_syntheticeps8:.3f}")

Fidelity (Extracted Text vs Synthetic Sentence DP eps8): 0.575


In [10]:
#fidelity_extracted_syntheticeps8 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_eps8_v2')
fidelity_extracted_syntheticeps16 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_eps16')

#print(f"Fidelity (Extracted Text vs Synthetic Sentence DP eps8): {fidelity_extracted_syntheticeps8:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence DP eps16): {fidelity_extracted_syntheticeps16:.3f}")

Fidelity (Extracted Text vs Synthetic Sentence DP eps16): 0.539


In [12]:
fidelity_extracted_syntheticeps3 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_eps3')
fidelity_extracted_synthetice_nodp_temp1 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_nodp_temp1')

print(f"Fidelity (Extracted Text vs Synthetic Sentence DP eps3): {fidelity_extracted_syntheticeps3:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence no DP Temp 1 ): {fidelity_extracted_synthetice_nodp_temp1:.3f}")

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

/home/ray/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Fidelity (Extracted Text vs Synthetic Sentence DP eps3): 0.524
Fidelity (Extracted Text vs Synthetic Sentence no DP Temp 1 ): 0.523


In [9]:
df.columns

Index(['transcription', 'medical_specialty', 'age_related_sentence',
       'extracted_text', 'med_masked_transcriptions',
       'pii_masked_transcriptions', 'synthetic_sentence_nodp',
       'synthetic_sentence2', 'synthetic_sentence_dp_nobossep',
       'synthetic_sentence_dp_eps16', 'synthetic_sentence_dp_eps8_v2',
       'synthetic_sentence_dp_eps3', 'synthetic_sentence_nodp_temp1'],
      dtype='object')

In [21]:
fidelity_extracted_synthetic = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_nobossep')

In [22]:
fidelity_extracted_synthetic

0.7533589751531796

## Medical Data Fidelity Calculation

In [ ]:
# Calculate fidelity for various comparisons
fidelity_extracted_extracted = calculate_fidelity(df, 'extracted_text', 'extracted_text')
fidelity_extracted_pii_masked = calculate_fidelity(df, 'extracted_text', 'pii_masked_transcriptions')
fidelity_extracted_synthetic = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_nodp')
fidelity_extracted_syntheticeps8 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_eps8_v2')
fidelity_extracted_syntheticeps16 = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_eps16')
fidelity_extracted_synthetic_dp_nobossep = calculate_fidelity(df, 'extracted_text', 'synthetic_sentence_dp_nobossep')

# Print the results
print(f"Fidelity (Extracted Text vs Extracted Text): {fidelity_extracted_extracted:.3f}")
print(f"Fidelity (Extracted Text vs PII Masked Transcriptions): {fidelity_extracted_pii_masked:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence No DP): {fidelity_extracted_synthetic:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence DP eps8): {fidelity_extracted_syntheticeps8:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence DP eps16): {fidelity_extracted_syntheticeps16:.3f}")
print(f"Fidelity (Extracted Text vs Synthetic Sentence DP No BOS SEP): {fidelity_extracted_synthetic_dp_nobossep:.3f}")


## Example Calculation

In [ ]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

# Tokenize input texts
texts = [
    "There's a kid on a skateboard.",
    "A kid is skateboarding.",
    "A kid is inside the house."
]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output

# Calculate cosine similarities
# Cosine similarities are in [-1, 1]. Higher means more similar
cosine_sim_0_1 = 1 - cosine(embeddings[0], embeddings[1])
cosine_sim_0_2 = 1 - cosine(embeddings[0], embeddings[2])

print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[1], cosine_sim_0_1))
print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[2], cosine_sim_0_2))